In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

### Task: Implement Iterative Policy Evaluation
Since this is a kind of **dynamic programming** algorithm, a *complete model of the environment is required*!

##### Pseudocode:
```
1) Init V(s) and pi(s) randomly

Loop until diff < theta
    diff = 0
    
    for each s in S:
        v = V(s)
        

```


In [123]:
# Init the actions and the environment model
A = ['l', 'r', 'u', 'd'] # set of possible actions: left, right, up, down
A = dict(zip(A, [-1, +1, -4, +4]))
S = np.arange(16) # all states (grid world 4x4)
T = np.array([0, 15]) # terminal states

def valid_action(s, a):
    assert s in S
    assert a in A
    
    if s + A[a] in S:
        return True
    else:
        return False

# policy : probability of taking action a given state s Pr{a | s}
def pi(s, a):
    assert s in S
    assert a in A
    
    # since every action can be done in 3/4 of all states, the actions are all equally likely ?!
    return 1 / len(A)

def get_next_state(s, a):
    if s in T:
        return s
    
    if s + A[a] in S:
        return s + A[a]
    else:
        return s

def init_state_value_function():
    V = np.zeros(16)
    return V

def R(s, s_, a): # get reward when transitioning from s to s_ with action a
    if s_ in T or s in T:
        return 0.
    else:
        return -1.
    
def P(s, s_, a):
    if get_next_state(s, a) == s_:
        return 1.
    else:
        return 0.
    
def get_next_possible_states(s):
    S_ = []    
    for a in A:
        S_.append(get_next_state(s, a))
    return S_
    
def print_grid(S, s = ''):
    print(s)
    print(S.reshape(4, 4))
    
    
def calc_bellman_equation(current_state, V, gamma):
    V_s = 0
        
    for action in A:
        reward = 0.
        possible_next_states = get_next_possible_states(current_state)
        
        for possible_next_state in possible_next_states:            
            reward += P(current_state, possible_next_state, action) * (R(current_state, possible_next_state, action) + gamma * V[possible_next_state])
    
        V_s += pi(current_state, action) * reward
            
    return V_s    

In [126]:
def iterative_policy(pi, theta = .5, gamma=1., verbose=True):  
    V = init_state_value_function()
    V_old = V.copy()
    
    diff = 2
    t = 0
    print_grid(S, 'states')
    while (diff > theta):
        if verbose:
            print_grid(V, 'state-value function after t = %i' % (t))
        diff = 0
        for s in S:
            v = V[s]
            # bellmann
            Vs = 0
            for a in A:
                returnn = 0
                for s_ in get_next_possible_states(s):
                    returnn += P(s, s_, a) * (R(s, s_, a) + gamma * V_old[s_])
                Vs += pi(s, a) * returnn    
            # assert Vs == calc_bellman_equation(s, V_old, gamma)
            V[s] = Vs
            
            diff = max(diff, np.abs(v - V[s]))
        V_old = V.copy()
        t += 1
    
    return V, t
    

In [127]:
Vpi, t = iterative_policy(pi, 1e-5, verbose=False)

print_grid(Vpi, 'Final state-value function after t = %i ' % (t))

states
[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]]
Final state-value function after t = 197 
[[  0.         -12.27261839 -17.30287577 -17.6059027 ]
 [-13.72714947 -16.51500435 -18.03014164 -17.78771907]
 [-17.78771907 -18.03014164 -16.51500435 -13.72714947]
 [-17.6059027  -17.30287577 -12.27261839   0.        ]]
